# DAMA50: Mathematics for machine learning

## Written assignment 5

### Georgios Zavlanis


### Problem 7

a. First, we have to "translate" the problem into a linear programming problem. The process can be seen below:

We have 42 million dollars in total, for three types of investments.
- Let be $x_1$ for treasury notes, $x_2$ for bonds and $x_3$ for stocks.
- We have a minimum investment of 3 million dollars in each category, which means that $ x_1, x_2, x_3 \ge 3000000 $.
- Also, at least half of the total funds allocated to treasury notes and bonds, which means that $ x_1+x_2 \ge 21000000 $.
- Finally, the investment in bonds must not exceed twice the investment in treasury notes, which means that $ x_2 \le 2x_1 $ or $ x_2 - 2x_1 \le 0 $.

We need to maximize the returns, so we need max $ 0.07x_1 + 0.09x_2 + 0.1x_3 $.

b. So now, we are ready to solve the linear problem: